In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import sklearn as sk
import tensorflow_addons as tfa
import cv2 as cv

# data_preprocessing.py

In [21]:
def preprocess(ds, model_params, batch_size, ds_type='train'):
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    resize_and_rescale = create_resize_and_rescale_layer(model_params)
    data_augmentation = create_augmentation_layer()

    if ds_type == 'test':
        ds = ds.map(lambda ds: (ds['image'], ds['label']))

    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), num_parallel_calls=AUTOTUNE) # disable to visualise train/ valid images
    ds = ds.batch(batch_size)

    if ds_type == 'train':
        ds = ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE)
    
    if ds_type == 'train' or ds_type == 'valid': 
        ds = ds.map(lambda x, y: (x, tf.one_hot(y, depth=model_params['num_classes'])))
    
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


def ensemble_input(ds, ds_type='train'):
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    if ds_type == 'train' or ds_type == 'valid':
        ds = ds.map(lambda x, y: ({ 'ensemble_0_input_2': x, 'ensemble_1_input_2': x, 'ensemble_2_input_2': x}, y))
    if ds_type == 'test':
        ds = ds.map(lambda ds: ({ 'ensemble_0_input_2': ds['image'], 'ensemble_1_input_2': ds['image'], 'ensemble_2_input_2': ds['image']}, ds['label']))
    
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


def create_resize_and_rescale_layer(model_params): 
    resize_and_rescale = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(
            model_params['image_shape'][0], 
            model_params['image_shape'][1], 
            interpolation='bilinear'
        ),
        tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
    ], name='resize_and_rescale')
    return resize_and_rescale


def create_augmentation_layer():
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip(),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    ], name='data_augmentation')
    return data_augmentation


def make_generator(seed=None):
    if seed:
        return tf.random.Generator.from_seed(seed)
    else:
        return tf.random.Generator.from_non_deterministic_state()


def random_invert_img(x, p=0.5):
    if tf.random.uniform([]) < p:
        x = (255-x)
    else:
        x
    return x


class RandomInvert(tf.keras.layers.Layer):
    def __init__(self, factor=0.5, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor
    
    def call(self, x):
        return random_invert_img(x)



class RandomSheer(tf.keras.layers.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.rng = make_generator(seed)


    def call(self, x):
        x_level = self.rng.uniform(shape=[], minval=0.1, maxval=0.35)
        y_level = self.rng.uniform(shape=[], minval=0.1, maxval=0.35)

        x = tfa.image.shear_x(x, x_level, 0)
        x = tfa.image.shear_y(x, y_level, 0)
        return x


class RandomGaussionFilter(tf.keras.layers.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)


    def call(self, x):
        k = np.random.randint(3, 11)
        sigma = np.random.uniform(low=0.1, high=0.9)
        x = tfa.image.gaussian_filter2d(x, filter_shape=(k, k), sigma=sigma)
        return x


# utils.py

In [ ]:
def plot(acc, val_acc, loss, val_loss, initial_epochs=0):
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1.0])
    plt.title('Training and Validation Accuracy')
    if initial_epochs != 0:
        plt.plot([initial_epochs-1,initial_epochs-1],
            plt.ylim(), label='Start Fine Tuning')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0, 1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    if initial_epochs != 0:
        plt.plot([initial_epochs-1,initial_epochs-1],
            plt.ylim(), label='Start Fine Tuning')
    plt.show()


def save_results(image_ids, predicted_labels, save_path):
    results = image_ids.drop('image', axis=1)
    results.columns = ['ID', 'Label']
    results['Label'] = predicted_labels 
    results = results.sort_values('ID').reset_index(drop=True)
    results.to_csv(save_path, index=False)

# model.py

In [3]:
class AdaptiveConcatPooling(tf.keras.layers.Layer):
    def __init__(self):
        super(AdaptiveConcatPooling, self).__init__()

    
    def call(self, x):
        output_size = (x.shape[1], x.shape[2])
        avg_pool = tfa.layers.AdaptiveAveragePooling2D(output_size)(x)
        max_pool = tfa.layers.AdaptiveMaxPooling2D(output_size)(x)
        return tf.concat([avg_pool, max_pool], axis=1)


def create_base_model(model_type, img_shape):
    if model_type == 'mobile':
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=img_shape,
            include_top=False,
            weights='imagenet'
        )
    elif model_type == 'xception':
        base_model = tf.keras.applications.Xception(
            input_shape=img_shape,
            include_top=False,
            weights='imagenet',
        )
    elif model_type == 'res':
        base_model = tf.keras.applications.ResNet152V2(
            input_shape=img_shape,
            include_top=False,
            weights='imagenet',
        )
    else:
        raise RuntimeError(f'model_type={model_type} unsupported')

    base_model.trainable = False
    return base_model


def create_prediction_layer(head_type, num_classes, hyperparams):
    if head_type == 'standard':
        prediction = tf.keras.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(hyperparams['dropout']),
            tf.keras.layers.Dense(num_classes, name='logits'),
            tf.keras.layers.Activation('softmax', dtype='float32', name='probs'), # separate activation layer for mixed precision support 
        ], name='prediction')
    elif head_type == 'adaptive':
        prediction = tf.keras.Sequential([
            AdaptiveConcatPooling(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.75),
            tf.keras.layers.Dense(num_classes, name='logits'),
            tf.keras.layers.Activation('softmax', dtype='float32', name='probs'),
        ], name='prediction')
    else:
        raise RuntimeError(f'head_type={head_type} unsupported')
    
    return prediction


def create_model(image_shape, num_classes, hyperparams, model_type='mobile', head_type='standard'):
    base_model = create_base_model(model_type, image_shape)
    prediction_layer = create_prediction_layer(head_type, num_classes, hyperparams)

    inputs = tf.keras.Input(shape=image_shape)
    x = base_model(inputs, training=False)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    return model


def create_fine_tune_model(model, hyperparams):
    print("Number of layers in the base model: ", len(model.layers[1].layers))
    model.layers[1].trainable = True
    for layer in model.layers[1].layers[:hyperparams['fine_tune_at']]:
        layer.trainable = False

    return model


def load_ensmble_models():
    filenames = ['Xception09.1-1_fold', 'MobileNet03-1_fold', 'ResNet03-1_fold']
    models = []
    for i, name in enumerate(filenames):
        model = tf.keras.models.load_model(os.path.join('models', name, '.h5'))
        for layer in model.layers:
            layer.trainable = False
            layer._name = f'ensemble_{i}_{layer.name}'
        models.append(model)
    return models


def create_ensemble_prediction_layer():
    prediction = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(3, name='logits'),
        tf.keras.layers.Activation('softmax', dtype='float32', name='probs'),
    ], name='prediction')
    return prediction

    
def create_ensemble_model():
    models = load_ensmble_models()
    prediction_layer = create_ensemble_prediction_layer()
    
    ensemble_inputs = [model.input for model in models]
    ensemble_outputs = [model.output for model in models]
    concat = tf.keras.layers.concatenate(ensemble_outputs)
    outputs = prediction_layer(concat)
    model = tf.keras.Model(inputs=ensemble_inputs, outputs=outputs)
    return model
    

# train.py

In [4]:
def train_validate(model, train_ds, valid_ds, hyperparams, initial_epoch, num_epochs, callbacks):    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hyperparams['learning_rate']
        ),
        loss=tf.keras.losses.CategoricalCrossentropy(
            label_smoothing=0.1
        ),
        metrics=['accuracy']
    )
    history = model.fit(
        train_ds,
        validation_data=valid_ds,
        initial_epoch=initial_epoch,
        epochs=num_epochs,
        callbacks=callbacks,
    )
    return model, history


def feature_extract_and_fine_tune(experiment_name, train_ds, valid_ds, model_params, base_hyperparams, fine_hyperparams):
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join('logs', experiment_name)
    )

    tf.keras.backend.clear_session()
    model = create_model(model_params['image_shape'], model_params['num_classes'], base_hyperparams)
    model, history = train_validate(
        model=model, 
        train_ds=train_ds, 
        valid_ds=valid_ds, 
        hyperparams=base_hyperparams, 
        initial_epoch=0, 
        num_epochs=base_hyperparams['num_epochs'],
        callbacks=[tensorboard_callback]
    )

    model = create_fine_tune_model(model, fine_hyperparams)
    model, history = train_validate(
        model=model, 
        train_ds=train_ds, 
        valid_ds=valid_ds, 
        hyperparams=fine_hyperparams, 
        initial_epoch=base_hyperparams['num_epochs'], 
        num_epochs=base_hyperparams['num_epochs'] + fine_hyperparams['num_epochs'],
        callbacks=[tensorboard_callback]
    )
    return model


def cross_validate(experiment_name, train_folds, valid_folds, model_params, base_hyperparams, fine_hyperparams):
    models = []
    train_accs, valid_accs, train_losses, valid_losses = [], [], [], []
    for i, (train_ds, valid_ds) in enumerate(zip(train_folds, valid_folds)):
        k = i + 1
        experiment_name_fold = f'{experiment_name}: {k}-fold'
        print(f'# -------------------- {experiment_name_fold} -------------------- #')
        
        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join('logs', experiment_name_fold)
        )

        tf.keras.backend.clear_session()
        model = create_model(model_params['image_shape'], model_params['num_classes'], base_hyperparams)
        model, history = train_validate(
            model=model, 
            train_ds=train_ds, 
            valid_ds=valid_ds, 
            hyperparams=base_hyperparams, 
            initial_epoch=0, 
            num_epochs=base_hyperparams['num_epochs'],
            callbacks=[tensorboard_callback]
        )

        model = create_fine_tune_model(model, fine_hyperparams)
        model, history = train_validate(
            model=model, 
            train_ds=train_ds, 
            valid_ds=valid_ds, 
            hyperparams=fine_hyperparams, 
            initial_epoch=base_hyperparams['num_epochs'], 
            num_epochs=base_hyperparams['num_epochs'] + fine_hyperparams['num_epochs'],
            callbacks=[tensorboard_callback]
        )

        train_acc = history.history['accuracy'][-1]
        valid_acc = history.history['val_accuracy'][-1]
        train_loss = history.history['loss'][-1]
        valid_loss = history.history['val_loss'][-1]

        print(f'{experiment_name} | Train Loss: {train_loss} | Train Accuracy: {train_acc} | Validation Loss: {valid_loss} | Validation Accuracy: {valid_acc}\n')
        
        models.append(model)
        train_accs.append(train_acc)
        valid_accs.append(valid_acc)
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

    avg_train_acc = np.mean(train_accs)
    avg_valid_acc = np.mean(valid_accs)
    avg_train_loss = np.mean(train_loss)
    avg_valid_loss = np.mean(valid_loss)

    print(f'Avg Train Loss: {avg_train_loss} | Avg Train Accuracy: {avg_train_acc} | Avg Validation Loss: {avg_valid_loss} | Avg Validation Accuracy: {avg_valid_acc}\n')
    
    return models


def evaluate(model, test_ds):
    predictions = model.predict(test_ds)
    predicted_indices = tf.argmax(predictions, 1)
    predicted_labels = predicted_indices.numpy()
    return predicted_labels

# main.py

In [5]:
# pre pre-process images
# python preprocess.py

# Build dataset
# python -m tensorflow_datasets.scripts.download_and_prepare --datasets=mri_dataset --module_import=datasets.mri_dataset --manual_dir=data/processed --data_dir=data/

In [6]:
# def hist_norm(img):
#     gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     norm_gray_img = cv.equalizeHist(gray_img)
#     norm_img = cv.cvtColor(norm_gray_img, cv.COLOR_GRAY2RGB)
#     return norm_img


# def clahe(img, clipLimit=4, tileGridSize=(40, 40)):
#     clahe = cv.createCLAHE(clipLimit, tileGridSize)
#     gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     cl_img = clahe.apply(gray_img)
#     cl_img = cv.cvtColor(cl_img, cv.COLOR_GRAY2RGB)
#     return cl_img


# data_folder = 'data'
# raw_folder = 'raw'
# processed_folder = 'processed'
# dataset = 'mri_dataset'
# train_folder = 'train'
# test_folder = 'test'
# train_label = 'train_label.csv'

# img_path = os.path.join(data_folder, raw_folder, train_folder, '851.png')
# img = cv.imread(img_path)
# n_img = hist_norm(img)
# clahe_img1 = clahe(img, clipLimit=8, tileGridSize=(32, 32))
# clahe_img2 = clahe(img, clipLimit=8, tileGridSize=(8, 8))

# # n_clahe_img = clahe(n_img, clipLimit=clip, tileGridSize=tile_size)
# combined_imgs = np.hstack((img, clahe_img1, clahe_img2))
# plt.figure(figsize=(20, 20))
# plt.imshow(combined_imgs)

## GPU Setup

In [7]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable first GPU
  tf.config.set_visible_devices(physical_devices[1:], 'GPU')
  logical_devices = tf.config.list_logical_devices('GPU')
  # Logical device was not created for first GPU
  assert len(logical_devices) == len(physical_devices) - 1
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

## Enable Mixed Precision

In [8]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

Some of your GPUs may run slowly with dtype policy mixed_float16 because they do not all have compute capability of at least 7.0. Your GPUs:
  Tesla V100S-PCIE-32GB, compute capability 7.0
  Tesla V100-PCIE-32GB, compute capability 7.0
  Tesla P100-PCIE-16GB, compute capability 6.0 (x2)
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


## Load Dataset

In [9]:
import tensorflow_datasets as tfds
from datasets.mri_dataset import MriDataset
from tensorflow.data import Dataset
from tensorflow.keras.callbacks import TensorBoard


data_folder = 'data'
raw_folder = 'raw'
processed_folder = 'processed'
dataset = 'mri_dataset'
train_folder = 'train'
test_folder = 'test'
train_label = 'train_label.csv'

SEED = 0

experiment_name = 'MobileNetV2'
model_params = {
    'image_shape': (512, 512, 3),
    'num_classes': 3,
}
base_hyperparams = {
    'train_batch_size': 64,
    'valid_batch_size': 64,
    'test_batch_size': 64,
    'num_epochs': 1,
    'learning_rate': 1e-4,
    'dropout': 0.2
}
fine_hyperparams = {
    'num_epochs': 1,
    'learning_rate': 1e-5,
    'fine_tune_at': 100,
}

tf.random.set_seed(SEED)

train_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[:{k}%]+train[{k+10}%:]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
valid_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[{k}%:{k+10}%]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
test_ds_raw, test_info_raw = tfds.load(
    name='mri_dataset', 
    split='test', 
    download=False, 
    shuffle_files=False, 
    as_supervised=False, 
    with_info=True,
    data_dir=data_folder
)

train_folds = [ preprocess(ds, model_params, batch_size=base_hyperparams['train_batch_size'], ds_type='train') for ds in train_folds ]
valid_folds = [ preprocess(ds, model_params, batch_size=base_hyperparams['valid_batch_size'], ds_type='valid') for ds in valid_folds ]
test_ds = preprocess(test_ds_raw, model_params, batch_size=base_hyperparams['test_batch_size'], ds_type='test')

In [10]:
train_valid_df = pd.read_csv(os.path.join(data_folder, processed_folder, train_label))
train_ds = train_folds[0]
valid_ds = valid_folds[0]

print(f'Number of train batches: {train_ds.cardinality()}')
print(f'Number of valid batches: {valid_ds.cardinality()}')
print(f'Number of test batches: {test_ds.cardinality()}')

Number of train batches: 17
Number of valid batches: 2
Number of test batches: 5


## Visualise Dataset

In [11]:
# # disable image resize and rescale in preprocess function ONLY for visualisation
# # Train Data
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(9):
#     for i in range(9):
#       ax = plt.subplot(3, 3, i + 1)
#       plt.imshow(images[i].numpy().astype("uint8"))
#       plt.title(labels[i].numpy())
#       plt.axis("off")

In [12]:
# # Test Data
# plt.figure(figsize=(10, 10))
# for i, ds in enumerate(test_ds_raw.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(ds['image'].numpy().astype("uint8"))
#     plt.title('ID: {}'.format(ds['id'].numpy()))
#     plt.axis("off")

In [13]:
# TODO: Fix visualisation
# data_augmentation = create_augmentation_layer()

# for image, _ in train_ds.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

## Train and Validate

### Feature Extraction + Fine Tuning

In [14]:
tf.random.set_seed(SEED)

model = feature_extract_and_fine_tune(experiment_name, train_ds, valid_ds, model_params, base_hyperparams, fine_hyperparams)

 1/17 [>.............................] - ETA: 0s - loss: 1.3564 - accuracy: 0.1719WARNING:tensorflow:From /home/zongsien/MedicalImageClassifier/env/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
17/17 [==============================] - 9s 540ms/step - loss: 1.2518 - accuracy: 0.2710 - val_loss: 1.0954 - val_accuracy: 0.4138
Number of layers in the base model:  155
Epoch 2/2
17/17 [==============================] - 5s 280ms/step - loss: 0.9306 - accuracy: 0.5754 - val_loss: 0.7640 - val_accuracy: 0.6983


### K-Fold Cross Validation

In [15]:
tf.random.set_seed(SEED)

models = cross_validate(experiment_name, train_folds, valid_folds, model_params, base_hyperparams, fine_hyperparams)
#     print('Saving model\n')
#     filename = os.path.join('models', experiment_name, '.h5')
#     model.save(filename)

ain Loss: 0.9243298768997192 | Train Accuracy: 0.569656491279602 | Validation Loss: 0.7638368010520935 | Validation Accuracy: 0.6982758641242981

# -------------------- MobileNetV2: 2-fold -------------------- #
17/17 [==============================] - 3s 203ms/step - loss: 1.1088 - accuracy: 0.4494 - val_loss: 1.1113 - val_accuracy: 0.4569
Number of layers in the base model:  155
Epoch 2/2
17/17 [==============================] - 5s 299ms/step - loss: 0.8602 - accuracy: 0.6279 - val_loss: 0.7770 - val_accuracy: 0.6638
MobileNetV2 | Train Loss: 0.8602254986763 | Train Accuracy: 0.6278625726699829 | Validation Loss: 0.7769647836685181 | Validation Accuracy: 0.6637930870056152

# -------------------- MobileNetV2: 3-fold -------------------- #
17/17 [==============================] - 4s 215ms/step - loss: 1.1988 - accuracy: 0.3254 - val_loss: 1.1870 - val_accuracy: 0.3966
Number of layers in the base model:  155
Epoch 2/2
17/17 [==============================] - 4s 264ms/step - loss: 0.89

In [16]:
# tf.random.set_seed(SEED)
# hyperparams = {
#     'initial_epochs': 150,
#     'learning_rate': 1e-4,
#     'label_smoothing': 0.1,
# }
# experiment_name = f'Ensemble02'


# ensemble_model = create_ensemble_model()
# train_ds = ensemble_input(train_folds[3], ds_type='train')
# valid_ds = ensemble_input(valid_folds[3], ds_type='valid')
# ensemble_model = train_validate(ensemble_model, train_ds, valid_ds, hyperparams, experiment_name)


In [17]:
# test_ds = ensemble_input(test_ds, ds_type='train')
# predictions = ensemble_model.predict(test_ds)
# predicted_indices = tf.argmax(predictions, 1)
# predicted_labels = predicted_indices.numpy()
# img_ids = tfds.as_dataframe(test_ds_raw, test_info_raw)
# save_results('Ensemble02_submission.csv', img_ids, predicted_labels)
# print('done')

## Predict

In [18]:
# TODO: analyse predicted results
# image_batch, label_batch = valid_ds.as_numpy_iterator().next()
# predictions = model.predict_on_batch(image_batch)
# predicted_indices = tf.argmax(predictions, 1)
# predicted_labels = predicted_indices.numpy()


# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   plt.title(f'pred: {predicted_labels[i]} true: {label_batch[i]}')
#   plt.axis("off")

## Evaluate

In [19]:
# predict test labels
predicted_labels = evaluate(model, test_ds)
predicted_labels

array([2, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 2,
       0, 1, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1,
       2, 1, 1, 2, 2, 0, 1, 0, 2, 0, 2, 1, 0, 1, 2, 1, 0, 1, 0, 0, 2, 1,
       1, 1, 2, 0, 0, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 1, 2, 1,
       1, 1, 2, 1, 1, 1, 2, 0, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2,
       1, 0, 2, 0, 1, 2, 0, 1, 2, 1, 0, 1, 2, 1, 1, 0, 1, 2, 0, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 2,
       2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 0, 1,
       0, 1, 0, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2,
       1, 2, 1, 1, 1, 1])

In [22]:
result_save_path = experiment_name + '_' + 'submission.csv'
img_ids = tfds.as_dataframe(test_ds_raw, test_info_raw)
save_results(img_ids, predicted_labels, result_save_path)